In [1]:
from keras.models import Sequential, load_model, Input, Model
from keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D,Dropout, Reshape, LSTM, BatchNormalization,TimeDistributed
from keras.layers import Conv2D, GlobalAveragePooling2D
from keras.optimizers import Adam #, RMSprop, SGD
from keras.utils import np_utils
from keras import regularizers
from keras.backend import random_normal
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from time import time

import pickle
import random
import numpy as np
import sys
import h5py
import glob
import os

Using TensorFlow backend.


In [2]:
batch_size =  16 #multiple of 1290/5 (recording length)
nb_location_classes = 6
nb_scene_classes=10
rows, cols = 431, 128   
nb_epoch = 200
pool_size = (5,5)                  # size of pooling area for max pooling
prob_drop_conv = 0.3                # drop probability for dropout @ conv layer
prob_drop_hidden = 0.3              # drop probability for dropout @ fc layer

In [3]:
#might need to rewrite these



def fileLists():
    trainlist=[]
    validationlist=[]
    testlist=[]
    evalSetupFiles='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\evaluation_setup\\*.txt'
    txtfilelist=glob.glob(evalSetupFiles)
    for txt in txtfilelist:
        if '_location' not in txt:
            continue
        with open(txt,'r') as evaltxtfile:
            for line in evaltxtfile.readlines():
                line=line.strip().split('\t')[0]
                if 'train' in txt:
                    trainlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                elif 'test' in txt:
                    testlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                else:
                    validationlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
    print('trainfiles: ', str(len(trainlist)))
    print('validationfiles: ', str(len(validationlist)))
    print('testfiles: ', str(len(testlist)))
    return trainlist,validationlist,testlist

labelRef={'barcelona':0, 'helsinki':1, 'london':2, 'paris':3, 'stockholm':4, 'vienna':5,
        'airport':0, 'shopping_mall':1, 'metro_station':2,'street_pedestrian':3,'public_square':4,
        'street_traffic':5, 'tram':6, 'bus':7, 'metro':8, 'park':9}

def getData(flist, nb_t, cpt):
    pth='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\'
    X_=np.zeros(((len(flist)),128,431))
    Y_=[]
    for i in cpt:
        Y_.append(np.zeros(((len(flist)),i)))
    for i,tfile in enumerate(flist):
        with open(pth+tfile, "rb" ) as scenesample:
            fv=pickle.load(scenesample)
        X_[i,:,:]=fv
        scene=tfile.split('-')[0].replace('logMelSpec\\','')
        Y_[0][i,labelRef[scene]]=1
        location=tfile.split('-')[1]
        Y_[1][i,labelRef[location]]=1
    return X_, Y_
trainlist,validationlist,testlist=fileLists()
nb_tasks=2
classespertask=[nb_scene_classes,nb_location_classes]
#X_train, Y_train = getData(trainlist,nb_tasks,classespertask)
#X_val, Y_val = getData(validationlist)
#X_test, Y_test = getData(testlist)

print(' data sorted')


trainfiles:  6122
validationfiles:  2518
testfiles:  2518
 data sorted


In [44]:
###### Convolutional model

def convBNpr(inputlayer,filters,kernalsize):
    outputlayer=Conv2D(filters, kernalsize, padding='same', activation='relu')(inputlayer)
    outputlayer=BatchNormalization()(outputlayer)
    return outputlayer

def compileCRNN(rows,cols,nb_scene_classes,nb_location_classes):
    a = Input(shape=(rows,cols,1))
    # CNN
    c1 = convBNpr(a, 32, (7,7))
    p1 = MaxPooling2D(pool_size=pool_size)(c1)
    d1 = Dropout(prob_drop_conv)(p1)
    c2 = convBNpr(d1, 64, (7,7))
    p2 = MaxPooling2D(pool_size=pool_size)(c2)
    d2 = Dropout(prob_drop_conv)(p2)
        
    # scene labels
    a1 = GlobalAveragePooling2D()(d2)
    out1 = Dense(nb_scene_classes, activation='sigmoid', kernel_regularizer=regularizers.l2(0.001))(a1)
    #, bias_initializer=random_normal(mean=-4.60, stddev=0.05, seed=123)
    # location labels
    a2 = GlobalAveragePooling2D()(d2)
    out2 = Dense(nb_location_classes, activation='sigmoid', kernel_regularizer=regularizers.l2(0.001))(a2)
    # , bias_initializer=random_normal(mean=-4.60, stddev=0.05, seed=123)    
    model = Model(inputs=a, outputs=[out1, out2])
    model.summary()
    model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], loss_weights=[0.5, 0.3], optimizer=Adam(lr=1e-5), metrics=['accuracy'])
    
    return model

model = compileCRNN(rows,cols,nb_scene_classes, nb_location_classes)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           (None, 431, 128, 1)  0                                            
__________________________________________________________________________________________________
conv2d_35 (Conv2D)              (None, 431, 128, 32) 1600        input_18[0][0]                   
__________________________________________________________________________________________________
batch_normalization_35 (BatchNo (None, 431, 128, 32) 128         conv2d_35[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_35 (MaxPooling2D) (None, 86, 25, 32)   0           batch_normalization_35[0][0]     
__________________________________________________________________________________________________
dropout_35

In [50]:
# Train
#fold=1
def train_network(model, model_name, X_train, Y_train, X_val, Y_val, nb_epoch, validationsplit_size, batchsize, early_stoping_patience, output_folder):
    checkpointer = ModelCheckpoint(filepath=os.path.join(output_folder,model_name + '.hdf5'),save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=early_stoping_patience)
    tensorboard = TensorBoard(log_dir="logs\\{}".format(time()))
    Callbacks=[checkpointer,  tensorboard] #early_stopping,
  #  print(samples)
    steps=int(samples/batchsize)
    validationsteps=int(validationsplit_size/batchsize)
    history = model.fit(X_train, Y_train, epochs=nb_epoch, callbacks=Callbacks, batch_size=batch_size, validation_data=(X_val, Y_val), shuffle=True, verbose=1)
    return history,model

def buildModel(savemodelfilename, samples,model,X_train, Y_train,X_val, Y_val):
    valSplit_size = int(samples/4)
    early_stoping_patience=5
    history,model = train_network(model, 'models\\multi_task5-3_400', X_train, Y_train, X_val, Y_val, nb_epoch, valSplit_size, batch_size, early_stoping_patience,'.')
    model.save_weights(savemodelfilename)
    return model, history


In [51]:
from datetime import datetime
batch_size =  16 #multiple of 1290/5 (recording length)
rows, cols = 431, 128  
nb_location_classes = 6
nb_scene_classes=10
nb_classes=16
nb_tasks=2
classespertask=[nb_scene_classes,nb_location_classes]
nb_epoch=100

#trainlist,validationlist,testlist=fileLists()
print('lists sorted')
#X_tr, Y_train= getData(trainlist,nb_tasks,classespertask)
#X_v, Y_val = getData(validationlist,nb_tasks,classespertask)
#X_te, Y_test = getData(testlist,nb_tasks,classespertask)
print('data obtained')
#X_train=np.expand_dims(X_tr,axis=3)
#X_val=np.expand_dims(X_v,axis=3)
#X_test=np.expand_dims(X_te,axis=3)
print('data sorted')
#model = compileCRNN(cols,rows,nb_scene_classes, nb_location_classes)
model=load_model('models\\multi_task5-3_300.hdf5')
print('model compiled')
savemodelfilename='models\\multi_task5-3_400.testsave'
samples=(X_train.shape[0])
print(datetime.now())
models,histories= buildModel(savemodelfilename, samples,model,X_train,Y_train,X_val, Y_val) #double Xs, one per scenes, t'other locations
print(datetime.now())
print('model saved')

lists sorted
data obtained
data sorted
model compiled
2019-04-11 21:42:30.827699
Train on 6122 samples, validate on 2518 samples
Epoch 1/100
6122/6122 [==============================] - 13s 2ms/step - loss: 0.5089 - dense_29_loss: 0.5761 - dense_30_loss: 0.6121 - dense_29_acc: 0.8131 - dense_30_acc: 0.7996 - val_loss: 1.2294 - val_dense_29_loss: 1.6012 - val_dense_30_loss: 1.3052 - val_dense_29_acc: 0.5624 - val_dense_30_acc: 0.5719
Epoch 2/100
6122/6122 [==============================] - 12s 2ms/step - loss: 0.5195 - dense_29_loss: 0.5928 - dense_30_loss: 0.6194 - dense_29_acc: 0.8079 - dense_30_acc: 0.7945 - val_loss: 1.2251 - val_dense_29_loss: 1.6356 - val_dense_30_loss: 1.2334 - val_dense_29_acc: 0.4873 - val_dense_30_acc: 0.5576
Epoch 3/100
6122/6122 [==============================] - 12s 2ms/step - loss: 0.5190 - dense_29_loss: 0.5889 - dense_30_loss: 0.6242 - dense_29_acc: 0.8061 - dense_30_acc: 0.7834 - val_loss: 1.2900 - val_dense_29_loss: 1.6760 - val_dense_30_loss: 1.3823 -

6122/6122 [==============================] - 12s 2ms/step - loss: 0.4960 - dense_29_loss: 0.5621 - dense_30_loss: 0.5890 - dense_29_acc: 0.8216 - dense_30_acc: 0.7963 - val_loss: 1.2090 - val_dense_29_loss: 1.5994 - val_dense_30_loss: 1.2367 - val_dense_29_acc: 0.4932 - val_dense_30_acc: 0.5540
Epoch 27/100
6122/6122 [==============================] - 12s 2ms/step - loss: 0.4959 - dense_29_loss: 0.5606 - dense_30_loss: 0.5911 - dense_29_acc: 0.8244 - dense_30_acc: 0.8037 - val_loss: 1.2018 - val_dense_29_loss: 1.5727 - val_dense_30_loss: 1.2572 - val_dense_29_acc: 0.5242 - val_dense_30_acc: 0.5413
Epoch 28/100
6122/6122 [==============================] - 12s 2ms/step - loss: 0.4908 - dense_29_loss: 0.5516 - dense_30_loss: 0.5889 - dense_29_acc: 0.8273 - dense_30_acc: 0.8025 - val_loss: 1.1433 - val_dense_29_loss: 1.4761 - val_dense_30_loss: 1.2228 - val_dense_29_acc: 0.5413 - val_dense_30_acc: 0.5846
Epoch 29/100
6122/6122 [==============================] - 12s 2ms/step - loss: 0.4885 

6122/6122 [==============================] - 12s 2ms/step - loss: 0.4713 - dense_29_loss: 0.5269 - dense_30_loss: 0.5617 - dense_29_acc: 0.8396 - dense_30_acc: 0.8131 - val_loss: 1.2452 - val_dense_29_loss: 1.6351 - val_dense_30_loss: 1.2945 - val_dense_29_acc: 0.4873 - val_dense_30_acc: 0.5413
Epoch 53/100
6122/6122 [==============================] - 12s 2ms/step - loss: 0.4727 - dense_29_loss: 0.5311 - dense_30_loss: 0.5592 - dense_29_acc: 0.8352 - dense_30_acc: 0.8166 - val_loss: 1.2116 - val_dense_29_loss: 1.6362 - val_dense_30_loss: 1.1804 - val_dense_29_acc: 0.4619 - val_dense_30_acc: 0.5651
Epoch 54/100
6122/6122 [==============================] - 12s 2ms/step - loss: 0.4719 - dense_29_loss: 0.5239 - dense_30_loss: 0.5686 - dense_29_acc: 0.8355 - dense_30_acc: 0.8117 - val_loss: 1.1884 - val_dense_29_loss: 1.5091 - val_dense_30_loss: 1.3148 - val_dense_29_acc: 0.5254 - val_dense_30_acc: 0.4889
Epoch 55/100
6122/6122 [==============================] - 12s 2ms/step - loss: 0.4751 

6122/6122 [==============================] - 12s 2ms/step - loss: 0.4561 - dense_29_loss: 0.5066 - dense_30_loss: 0.5415 - dense_29_acc: 0.8386 - dense_30_acc: 0.8171 - val_loss: 1.1901 - val_dense_29_loss: 1.5843 - val_dense_30_loss: 1.1921 - val_dense_29_acc: 0.4905 - val_dense_30_acc: 0.5850
Epoch 79/100
6122/6122 [==============================] - 12s 2ms/step - loss: 0.4538 - dense_29_loss: 0.5031 - dense_30_loss: 0.5397 - dense_29_acc: 0.8411 - dense_30_acc: 0.8224 - val_loss: 1.1189 - val_dense_29_loss: 1.4601 - val_dense_30_loss: 1.1617 - val_dense_29_acc: 0.4754 - val_dense_30_acc: 0.5461
Epoch 80/100
6122/6122 [==============================] - 12s 2ms/step - loss: 0.4553 - dense_29_loss: 0.5027 - dense_30_loss: 0.5452 - dense_29_acc: 0.8471 - dense_30_acc: 0.8182 - val_loss: 1.1592 - val_dense_29_loss: 1.5222 - val_dense_30_loss: 1.1922 - val_dense_29_acc: 0.5191 - val_dense_30_acc: 0.6021
Epoch 81/100
6122/6122 [==============================] - 12s 2ms/step - loss: 0.4563 

In [52]:
path='models\\multi_task5-3_400.hdf5'
model=load_model(path)

In [53]:
evaluation=model.evaluate(X_test, Y_test, verbose=0,batch_size=16)
print(model.metrics_names)
print(evaluation)

['loss', 'dense_29_loss', 'dense_30_loss', 'dense_29_acc', 'dense_30_acc']
[1.0710134690862692, 1.390926786317439, 1.1169168568490704, 0.56235107229142267, 0.59451945990063637]


In [ ]:
f